# Analisi dati relazione 2, esperienza 4
_martedì 21 dicembre_


## Osservazioni sulle relazioni di laboratorio n. 1

### Osservazioni sui fit

Spesso ci affidiamo troppo al risultato del fit, trascurando l'effettivo risultato del fit, ovvero osservando i valori di $\chi^2/\text{ndf}$ e poi verificandone così la bontà.

Ipotizzziamo che il fit sia vero a priori e sia più affidabile del nostro stesso senso fisico, ignorando quindi eventuali effetti secondari che si potrebbero osservare. 

> Si può in effetti provare ad escludere alcuni punti da quelli utilizzati per esempi osservando che siano messi fuori dalla linea del fit.

### Osservazioni sul fattore di merito

Il fattore di merito $Q$ del circuito RLC si presentava come

$$
H(\omega) = \frac{R}{R+R_L + j\left(\omega L  \frac{1}{\omega C}\right)}
$$

che riscrivendo in termini di modulo diventa necessariamente

$$
|H(\omega)| = \frac{1}{\left(1+ \frac{R}{R_L}\right) + Q_{id}\left(\frac{\omega}{\omega_0}-\frac{\omega_0}{\omega}\right)}
$$

dove però il fattore di merito è un fattore di merito ideal, mantre osserviamo l'esistenza di un fattore di merito $Q_{reale}$ reale, che tiene conto delle caratteristiche di impedenza dell'induttanza, dove effettivamente il suo valore dipende da diversi fattori come

$$
Q_{reale} = \frac{\sqrt{L}}{R_{tot} \sqrt{C}}
$$

dove $R_{tot}$ dipende da tutte le resistenze presenti nel circuito.

> (Il modo di ovviare a questo problema forse lo possiamo trovare nel considerare $L_0$ come il valore di $L$ trovato non dai calcoli ma dal valore di $L_0$ misurato nel caso libero)

### Osservazioni sui risultati ottenuti

Utilizzare senso fisico per descrivere il fenomeno osservato.

Rileggendo la relazione osservo che in effetti non sono forniste esaustive giustificazioni riguardo al valore di $\mu_R \text{(Al)}$ ottenuto ($\mu_R\text{(Al)} = 0.446(53)$) non corrisponde al modello (materiale paramagnetico).

In [1]:
%jsroot on
gInterpreter->AddIncludePath("/usr/local/include");
gSystem->Load("/usr/local/lib/libErrorAnalysis");
gSystem->Load("/usr/local/lib/libLabTools");

#include<vector>
#include<cmath>
#include<iostream>
#include<fstream>
#include<string>

#include<TCanvas.h>
#include<TGraphErrors.h>
#include<TF1.h>
#include<TStyle.h>
#include<TAxis.h>
#include<TMath.h>
#include<TLatex.h>
#include<TLegend.h>

In [2]:
#include<ErrorAnalysis.h>
#include<LabTools.h>

In [3]:
graphset::init();

std::string pfix = "../dati/_zeroed/";
const int full_dataT_s = 8;
std::string all_data_table[] = {
    "dati_2.txt",  //> 0
    "dati_4.txt",  //> 1
    "dati_5.txt",  //> 2
    "dati_6.txt",  //> 3
    "dati_8.txt",  //> 4 
    "dati_10.txt", //> 5
    "dati_12.txt", //> 6
    "dati_20.txt"  //> 7
};

std::ifstream data_table;

## Commenti Prof. Schiavi su possibili modelli per analizzare le curve di riscaldamento della lampadina

Abbiamo dei dati che rappresentano delle curve di riscaldamento, dobbiamo individuare quali sono le quantità che misuriamo e quelle che possiamo individuare attraverso dei calcoli. 

#### Cosa abbiamo misurato?

Abbiamo ottenuto delle curve di dati di tensione in funzione del tempo. 

Mi sono venuti in mente più modi per scegliere il modello su come affrontare questi dati?

_Quantità da misurare?_

Ci interessa misurare una differenza di tensione $\Delta V$. 

### Considerazioni di tipo fisico

La lampadina scalda il PT100, che però prima passa attraverso il vetro, che però poi passa attraverso una bolla d'aria. 

Quantità da misurare: Variazione di tensione in funzione del tempo $\Delta V (\Delta t)$

Definisco queste qunatità come segue:
- scelgo tempo $t=0$ quando accendo la lampadina;
- scelgo $\Delta t$ tale che io possa rimanere in regime di irraggiamento

Lasciamo per ora indietro il discorso sull'incertezza.

Arbitrarietà:
- Scelta di $\Delta t$: come cambiano i risultati scegliendo un altro valore di questo intervallo?  
(NOTA: interessante potrebbe essere notare cosa succede se sceglo un $\Delta t$ con valori anche nell'intervallo del secondo regime.
- Sistematica basata sullo studio del risultato finale con diversi $\Delta t$.

### Esempio funzionale per lo studio dei modelli possibili

Procedo ad analizzare le misure effettuate ad una distanza di .05 m 

In [4]:
int table = 2; 
std::string name_data_t[8] = {"d = 2#times10^{-2}m","d = 4#times10^{-2}m",
                              "d = 5#times10^{-2}m","d = 6#times10^{-2}m"
                              "d = 8#times10^{-2}m","d = 10#times10^{-2}m",
                              "d = 12#times10^{-2}m","d = 20#times10^{-2}m"};

data_table.open((pfix + all_data_table[table]).c_str());
TCanvas *c1 = new TCanvas();
c1->SetGrid();
TGraphErrors *g = new TGraphErrors();

double t, vin, evin, range; 

for(int j=0; data_table >> t >> vin >> evin >> range; j++){
    g->SetPoint(j, t, vin);
    g->SetPointError(j, 0, 0.01);
}
g->Draw("Ap");
g->SetTitle((name_data_t[table] + ";Time [s]; #splitline{Voltage}{Serial reading [V]}").c_str());
c1->Draw();
data_table.close();

In [5]:
TCanvas *c3 = new TCanvas("","", 1000, 500);
graphset::setcanvas(c3,4,2);

for(int k=0; k<8; k++){
    ifstream *data = new ifstream((pfix + all_data_table[k]).c_str());
    TGraphErrors *g1 = new TGraphErrors();
    for(int j=0; (*data >> t >> vin >> evin >> range) && j<60; j++){
        g1->SetPoint(j, t, vin);
        g1->SetPointError(j, 0.1, 0.01);
    }
    c3->cd(k+1);
    graphset::setcanvas(c3->cd(k+1));
    g1->Draw("AP");
    g1->GetXaxis()->SetLimits(g1->GetX()[0], g1->GetX()[0] + 80);
    g1->SetTitle((all_data_table[k] + ";Time [s]; Voltage [V]").c_str());
    data->close();
}
c3->Draw();

## Come potremmo calcolare la variazione di tensione rispetto al tempo

### Modello #1 (linearizzazione sul primo fronte di discesa)

Considerando solo i primi punti rispetto a $t=0$, preso quando viene accesa la lampadina, testare un modello lineare che prevede un andamento del tipo 

$$
V_{\text{out}}(t) = \text{(param0)} \cdot t + \text{(param1)}
$$

dove quindi possiamo individuare l'effettiva velocità $dV/dt$ con cui la tensione cambia rispetto al tempo. A questo punto poi la trattazione procede.

#### Errore sulla distanza (modello #1)

L'errore sulla distanza è legato a diversi fattori: 


| Quantità | valore |
|---|---|
| allineamento del supporto con il righelllo |1mm |
| allineamento del righello con il segno sulla base del PT100 | 1mm |
| errore di parallasse per ogni allineamento | 2mm |
| errore sul possibile scivolamento durante le misure | 2mm |
| $\varepsilon_d~(\text{stat})~~~(1\sigma)$ | $6/\sqrt{3}$ mm |


L'errore sistamatico (offset) può essere dovuto a:

| Quantità | ipotesi valore |
|---|---|
| I segni per l'allineamento sulla base del PT100 potrebbero non essere perpendicolari al piano | 3mm |
| I segni per l'allineamento sulla base della lampadina e il suo supporto | 3mm |
| possibile disallineamento tra lampadina e PT100, quindi sottostima della distanza | - |
| $\varepsilon_d~(\text{sys})~~~(1\sigma)$ | $\geq6/\sqrt{3}$ mm |

#### Modello di curva per dipendenza come $d^{-2}$

Il modello utilizzato è un modello del tipo
$$
\frac{dV}{dt} = V' = \alpha\cdot(d-\gamma)^{-\beta}
$$

In [6]:
double errore_tensione = 0.0044;

TCanvas *c4 = new TCanvas("","",1000,400);
graphset::setcanvas(c4, 2,1);
double d[8] = {0.02, 0.04, 0.05, 0.06, 0.08, 0.1, 0.12, 0.2};
double dVdt[8];
double err_dVdt[8];

Color_t mk[8] = {kRed, kOrange, kGreen-5, kAzure, kViolet-8, kGray+1, kBlue-3, kSpring-6};
//Color_t mk[8] = {100, 95, 90, 84, 65, 225, 57, 223};

c4->cd(1);
graphset::setcanvas(c4->cd(1));

TLegend *l = new TLegend(0.2, 0.2, 0.6, 0.5);
l->SetHeader("#bf{MODELLO #1}");
l->SetBorderSize(0);
l->SetFillColorAlpha(kWhite, 0);

for(int k = 0; k<8; k++){
    TGraphErrors *g1 = new TGraphErrors();
    ifstream data((pfix + all_data_table[k]).c_str());
    std::string dist = all_data_table[k].substr(all_data_table[k].find("_")+1, 
                                                all_data_table[k].find(".")-all_data_table[k].find("_")-1);

    g1->SetName(("g1_"+dist).c_str());
    
    double t1, v1, ev1, range1;

    data >> t1 >> v1 >> ev1 >> range1;

    g1->SetPoint(0, 0, 0);
    g1->SetPointError(0, 0,  errore_tensione);
    const double t0 = t1; // zeroing data
    const double v0 = v1;

    for(int j = 1; (data >> t1 >> v1 >> ev1 >> range1) && (j<15); j++){
        
        if(k==0 && t1<16.5 && t1>15.5) continue; // tolto punto doppio per d=2cm
        if(k==2 && t1<35 && t1>34) continue;     // tolto punto doppio per d=5cm
        
        g1->SetPoint(j, t1-t0, v1-v0);
        g1->SetPointError(j, 0,  errore_tensione);
    }
    data.close();

    TF1 *mod1 = new TF1(("mod1_"+ dist).c_str(), "[0]*x+[1]", 0, 10); // Per cambiare intervallo fit
    graphset::setmarker(g1, mod1, mk[k], 21, 0.4);
    mod1->SetParameter(0, 0.1);

    // logs::print_mmsg(("processing graph" + all_data_table[k]).c_str());
    
    g1->Draw((k==0)? "ap":"p");
    g1->SetTitle("; Time after t=0 [s]; Voltage [V]");
    graphset::set_TGraphAxis(g1, "Voltage [V]", 1.25, "Time after t=0 [s]");
    g1->Fit(("mod1_"+ dist).c_str(), "QR"); // fit sui primi 10s
    l->AddEntry(("g1_"+ dist).c_str(), ("d=" + dist + "#times10^{-2} m").c_str());
    g1->GetXaxis()->SetLimits(-1, 20);

    std::cout << "[[" << mod1->GetParameter(0) << " +- " << mod1->GetParError(0) << "], [" 
              << d[k] << " +- " << 0.006/sqrt(3)*2 << "]]" << std::endl;

    // logs::print_stat(mod1);
    
    dVdt[k] = mod1->GetParameter(0);
    err_dVdt[k] = mod1->GetParError(0);
}
l->Draw();
std::cout << std::endl;

c4->cd(2);
graphset::setcanvas(c4->cd(2));
TGraphErrors *plot = new TGraphErrors();
for(int j=0; j<8; j++){
    // if(j==0 || j==1) continue;
    plot->SetPoint(j, d[j], abs(dVdt[j]));
    plot->SetPointError(j, 0.006/sqrt(3)*2, err_dVdt[j]);
}
plot->Draw("ap");
// plot->SetTitle(";Source distance [m];Voltage speed after 10s [V/s]");
graphset::set_TGraphAxis(plot, "Voltage speed after 10s [V/s]", 1, "Source distance [m]");

TF1 *fit_mod1 = new TF1("fit_mod1", "[0]*pow((x+[2]), [1])");
fit_mod1->SetParameters(1, -2, 0.0);


plot->Fit("fit_mod1");

logs::print_stat(fit_mod1);
std::cout << std::endl 
          << "d^( " << fit_mod1->GetParameter(1) << " +- " << fit_mod1->GetParError(1) << " )" << std::endl
          << "offset (sys): " << fit_mod1->GetParameter(2) << " +- " << fit_mod1->GetParError(2) << std::endl;

c4->Draw();


std::cout << "Compatibilità di 2: " << 
    stattools::compatible(-2, 0, fit_mod1->GetParameter(1), fit_mod1->GetParError(1)) << std::endl;
std::cout << "compatibilità di zero: " << // offset con stima (6/sqrt(3) mm): " <<
    stattools::compatible(0, 0, fit_mod1->GetParameter(2), fit_mod1->GetParError(2)) << std::endl;

[[-0.0692979 +- 0.00047058], [0.02 +- 0.0069282]]
[[-0.0488412 +- 0.000463925], [0.04 +- 0.0069282]]
[[-0.0501754 +- 0.0004605], [0.05 +- 0.0069282]]
[[-0.0308415 +- 0.000462026], [0.06 +- 0.0069282]]
[[-0.0244878 +- 0.000463654], [0.08 +- 0.0069282]]
[[-0.0160445 +- 0.000464699], [0.1 +- 0.0069282]]
[[-0.0127483 +- 0.00046547], [0.12 +- 0.0069282]]
[[-0.00536735 +- 0.000465452], [0.2 +- 0.0069282]]

 FCN=3.91406 FROM MIGRAD    STATUS=CONVERGED     147 CALLS         148 TOTAL
                     EDM=5.00765e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   5.1 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.84511e-04   3.60952e-05   1.42043e-07   4.08126e+01
   2  p1          -2.02888e+00   4.50765e-02  -1.17610e-04   2.37819e-02
   3  p2           3.46169e-02   1.10637e-02   4.37005e-05  -8.30522e-02

** CHI2 / NDF ( PROB. ) 3.91406 / 5 ( 0.561854 )


d^( -2.0288

### Modello #2 (valutazione della variazione senza modellizzare la curva)

Procediamo azzerando i valori della tensione a $t=0$ s quando la lampadina viene accesa, considerando poi un intervallo sensato (_vedere dopo_) possiamo leggere la misura della tensione, poi valutare quindi la variazione. 

Per scegliere un intervallo sensato bisogna effettuare alcuni ragionamenti di come si propaga il calore e di come quindi interagisce con il PT100, e come risponde lo strumento. 

All'accensione dalla lampadina la luce viaggia a velocità $c$ quindi possiamo di fatto considerare la sua interazione con il termometro al platino (PT100) immediata. Dopo un po' di tempo però sul il termometro interviene un secondo fattore che non permette una misura veritiera della temeperatura. Dentro alla bolla contenente il termometro infatti aria intrappolata si scalda e dopo un certo intervallo (impossibile determinarlo solo con pochi dati a disposizione) interviene e contribuisce alla variazione della temperatura letta dallo strumento. Quindi non osserviamo più solo una lettura dell'irraggiamento, ma osserviamo anche una misura della temperatura dell'ambiente circostante, che interviene in modo importante e principale dopo un certo intervallo $\Delta t_1$. misurando quindi la variazione di tensione entro $\Delta t_1$ possiamo di fatto trascurare questo fatto. Potrebbe essere interessante procedere invece ad evidenziare intervalli successivamente grandi $\Delta t_2$, $\Delta t_3$, ... per vedere come si può comportare il risultato finale. 

### Modello #3 (derivazione fine del modello #2)

Procedere in modo simile al #2 ma di fatto effettuare alcune considerazioni sugli errori ottenuti da alcune considerazioni sulla variazione effettiva della tensione sul tempo.

In [20]:
TCanvas *c6 = new TCanvas("","",1000,400);
graphset::setcanvas(c6, 2,1);

TCanvas* c7 = new TCanvas();
TGraphErrors *mod2_suc = new TGraphErrors();

double dV_mod2[8];
double err_dV_mod2[8];

double eval[23] = {10, 12, 13, 14, 15, 16, 17, 20, 25, 27.5, 30, 32.5, 35, 37.5, 40, 42.5, 45, 50, 55, 60, 65, 70, 75};

c6->cd(1);
graphset::setcanvas(c6->cd(1));

TLegend *l2 = new TLegend(0.2, 0.2, 0.6, 0.5);
l2->SetHeader("#bf{MODELLO #2}");
l2->SetBorderSize(0);
l2->SetFillColorAlpha(kWhite, 0);

//for(int h=0; h<23; h++){
     int h = 0;

    for(int k = 0; k<8; k++){
        TGraphErrors *g1 = new TGraphErrors();
        ifstream data((pfix + all_data_table[k]).c_str());
        std::string dist = all_data_table[k].substr(all_data_table[k].find("_")+1, 
                                                    all_data_table[k].find(".")-all_data_table[k].find("_")-1);

        g1->SetName(("g1_"+dist).c_str());
    
        double t1, v1, ev1, range1;

        data >> t1 >> v1 >> ev1 >> range1;

        g1->SetPoint(0, 0, 0);
        g1->SetPointError(0, 0, errore_tensione);
        const double t0 = t1; // zeroing data
        const double v0 = v1;
    
        for(int j=1; (data >> t1 >> v1 >> ev1 >> range1) && (j<90); j++){
            if(k==3 && t1<68 && t1>67.5) continue; // tolto punto doppio per d=6cm
            g1->SetPoint(j, t1-t0, v1-v0);
            g1->SetPointError(j, 0, errore_tensione);
        }
    
        TF1 *mod2 = new TF1(("mod2_"+ dist).c_str(), "[0]*x+[1]", eval[h]-2, eval[h]+2);
        graphset::setmarker(g1, mod2, mk[k], 21, 0.4);
    
        g1->Draw((k==0)? "ap":"p");
        g1->SetTitle("; Time after t=0 [s]; Voltage [V]");
        graphset::set_TGraphAxis(g1, "Voltage [V]", 1, "Time after t=0 [s]");
        g1->Fit(("mod2_"+ dist).c_str(), "QR"); // fit intorno 20s
        l2->AddEntry(("g1_"+ dist).c_str(), ("d=" + dist + "#times10^{-2} m").c_str());
        g1->GetXaxis()->SetLimits(-1, 120);
    
        dV_mod2[k] = mod2->Eval(eval[h]);
        // ERRORE??
    
        data.close();
    }
    l2->Draw();
    
    
    c6->cd(2);
    graphset::setcanvas(c6->cd(2));
    TGraphErrors *plot2 = new TGraphErrors();
    for(int j=0; j<8; j++){
        plot2->SetPoint(j, d[j], abs(dV_mod2[j]));
        plot2->SetPointError(j, 0.006/sqrt(3)*2, errore_tensione); //> se considero 5 mm in più a d=2cm, ok compatibilità con -2.
    }
    plot2->Draw("ap");
    plot2->SetTitle(";Source distance [m];Voltage speed after 10s [V/s]");
    graphset::set_TGraphAxis(plot2, "Voltage after 20s [V/s]", 1, "Source distance [m]");
    
    TF1 *fit_mod2 = new TF1("fit_mod1", "[0]*pow((x+[2]), [1])");
    fit_mod2->SetParameters(1, -2, 0.0);
    
    
    plot2->Fit("fit_mod1");
    
    logs::print_stat(fit_mod2);
    std::cout << std::endl << "time : " << eval[h] << std::endl
              << "d^( " << fit_mod2->GetParameter(1) << " +- " << fit_mod2->GetParError(1) << " )" << std::endl
              << "offset (sys): " << fit_mod2->GetParameter(2) << " +- " << fit_mod2->GetParError(2) << std::endl;
    
    mod2_suc->SetPoint(h, eval[h], fit_mod2->GetParameter(1));
    mod2_suc->SetPointError(h, 0, fit_mod2->GetParError(1));

    c6->Draw();
    c6->SaveAs("../fig/plot_mod2.svg");

    std::cout << "Compatibilità di 2: " << 
        stattools::compatible(-2, 0, fit_mod2->GetParameter(1), fit_mod2->GetParError(1)) << std::endl;
    std::cout << "compatibilità di zero: " << // offset con stima (6/sqrt(3) mm): " <<
        stattools::compatible(0, 0, fit_mod2->GetParameter(2), fit_mod2->GetParError(2)) << std::endl;
//}


/*
c7->cd();
mod2_suc->SetMarkerStyle(21);
mod2_suc->Draw("AP");
c7->Draw();*/

 FCN=2.97802 FROM MIGRAD    STATUS=CONVERGED     339 CALLS         340 TOTAL
                     EDM=1.36095e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           5.83678e-03   1.86119e-03   2.90823e-07   4.07790e-01
   2  p1          -1.31510e+00   1.77886e-01   2.13911e-05  -6.15397e-03
   3  p2          -1.85412e-02   1.08289e-02   3.21021e-06  -4.81091e-02

** CHI2 / NDF ( PROB. ) 2.97802 / 5 ( 0.703375 )


time : 10
d^( -1.3151 +- 0.177886 )
offset (sys): -0.0185412 +- 0.0108289
Compatibilità di 2: NON-COMPATIBILE: significativita' statistica  3.850210 σ
compatibilità di zero: COMPATIBILE


Info in <TCanvas::Print>: SVG file ../fig/plot_mod2.svg has been created


### Idea di modello #4 (proposta da portare al prof., divisione delle componenti separando fenomeno irraggiamento e agitazione termica)

Provare a modellizzare il comportamento lineare sulla coda dei dati, quindi poi effettivamente sottrarre questo "rumore" ai valori dei dati, quindi ottenendo un comportamento che potrebbe essere di saturazione (come effettivamente ci aspettiamo di avere per il fenomeno dell'irraggiamento) e quindi osservando una curva asintotica all'infinito. 

Poi procedere simili al modello proposto in #2 o in #3. 

In [18]:
// testing in .05 m setup
TCanvas *c5 = new TCanvas();
c5->SetGrid();

g->Draw("ap");

double xmin = 65;
double xmax = 160;

TF1 *mod4_subtract = new TF1("mod4_subtract", "[0]+[1]*x", xmin, xmax);
g->Fit("mod4_subtract", "R");

TGraphErrors *clean = (TGraphErrors*)g->Clone();
graphset::setmarker(clean, kGreen-1);
for(int j=0; j<g->GetN(); j++){
    clean->SetPoint(j, g->GetX()[j], 
                    (g->GetX()[j]<=xmin)? 
                    g->GetY()[j]:g->GetY()[j]+abs(mod4_subtract->Eval(xmin)-mod4_subtract->Eval(g->GetX()[j])));
    clean->SetPointError(j, 0, 0.014);
}

clean->Draw("p");

 FCN=135.734 FROM MIGRAD    STATUS=CONVERGED      42 CALLS          43 TOTAL
                     EDM=7.80791e-20    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -1.53473e+00   4.99262e-03   6.59293e-06   4.31094e-08
   2  p1          -7.76223e-03   4.36132e-05   3.00000e-06  -4.18744e-06


### Considerazioni sull'incertezza

Come possiamo fare a valutare il valore di errore sulla misura di tensione?

Abbiamo raccolto un set di dati che rappresentato il valore di tensione letta quando la lampadina era spenta e quando solo la strumentazione era messa in tensione. 

Eseguendo un histo, possiamo effettivamente vedere se i dati si dispongono secondo una gausssiana, e nel caso effettivamente misurare come valore di errore a $1\sigma$ la dev. std. dell'istogramma.

In [16]:
std::string rumore = "../dati/valutazione_rumore.txt";

TCanvas *c2 = new TCanvas("","",1000, 500);
graphset::setcanvas(c2,2,1);

TLegend *l3 = new TLegend(0.55, 0.75, 1, 0.95);
l3->SetHeader("#bf{Valutazione errore}");
l3->SetBorderSize(0);
l3->SetFillColorAlpha(kWhite, 0);

gStyle->SetOptStat(0);

double T, V, eV, crange;
std::ifstream file(rumore.c_str());
TH1D *hist = new TH1D("hist", "", 20, -25e-3, 35e-3);
TGraphErrors *ghist = new TGraphErrors();
ghist->SetName("ghist");
ghist->SetMarkerStyle(21);
ghist->SetMarkerSize(0.3);
for(int j=0; file >> T >> V >> eV >> range >> crange; j++){
    hist->Fill(V);
    ghist->SetPoint(j, T, V);
    ghist->SetPointError(j, 0, 0.001);
}

c2->cd(1);
graphset::setcanvas(c2->cd(1));
hist->Draw("HIST");
hist->GetYaxis()->SetRangeUser(0,250);
graphset::set_TGraphAxis(hist, "Entries", 1, "Voltage [V]");

c2->cd(2);
graphset::setcanvas(c2->cd(2), 1,2);
c2->cd(2)->cd(1);
c2->cd(2)->cd(1)->SetBottomMargin(0);
c2->cd(2)->cd(1)->SetLeftMargin(0.175);
ghist->Draw("ap");
graphset::set_TGraphAxis(ghist, "Voltage [V]", 0.65, "Time [s]");
ghist->GetYaxis()->SetTitleOffset(1.75);
TF1 *sin = new TF1("sin", "[0]+[1]*cos([2]*(x-[3]))");
sin->SetParameters(0.005, 0.06, 0.03, 50);
const double* params = sin->GetParameters();
ghist->Fit("sin");

c2->cd(1);

TH1D *hist_corrected = new TH1D("hist_corrected","",20, -25e-3, 35e-3);
hist_corrected->SetLineColor(kRed);
TFormula *sin_ev = new TFormula("","[0]+[1]*cos([2]*(x-[3]))");
sin_ev->SetParameters(params);

TGraphErrors *ghist_corrected = new TGraphErrors();
ghist_corrected->SetName("ghist_corrected");
ghist_corrected->SetLineColor(kGreen+2);
ghist_corrected->SetMarkerStyle(21);
ghist_corrected->SetMarkerSize(0.3);
ghist_corrected->SetMarkerColor(kGreen+2);

for(int j=0; j<ghist->GetN(); j++){
    hist_corrected->Fill(ghist->GetY()[j]-sin_ev->Eval(ghist->GetX()[j]));
    ghist_corrected->SetPoint(j, ghist->GetX()[j], ghist->GetY()[j]-sin_ev->Eval(ghist->GetX()[j]));
    ghist_corrected->SetPointError(j, 0, 0.001);
}
hist_corrected->Draw("SAME");

c2->cd(2)->cd(2);
c2->cd(2)->cd(2)->SetBottomMargin(0.24);
c2->cd(2)->cd(2)->SetLeftMargin(0.175);
graphset::set_TGraphAxis(ghist_corrected, "Voltage [V]", 0.65, "Time [s]");
ghist_corrected->GetXaxis()->SetTitleOffset(2.25);
ghist_corrected->GetYaxis()->SetTitleOffset(1.75);
ghist_corrected->Draw("AP");

std::cout << "RMS (pre) : " << hist->GetRMS() << " +- " << hist->GetRMSError() << std::endl
          << "RMS (post): " << hist_corrected->GetRMS() << " +- " << hist_corrected->GetRMSError() << std::endl;

/*
c2->cd(3);
graphset::setcanvas(c2->cd(3), 2,5);
std::ifstream *batch_div = new ifstream(rumore.c_str());
int b = 0;
double xminv[10] = {0.01, 0.01, 0, 0, 0, -0.02, -0.02, -0.02, -0.01, 0};
double xmaxv[10] = {0.03, 0.03, 0.03, 0.02, 0.01, 0.01, 0, 0, 0.01, 0.02};
bool isnotlast;
do{
    TH1D *h = new TH1D(("h_" + std::to_string(b)).c_str(), "", 20, xminv[b], xmaxv[b]);
    for(int i = 0; *batch_div >> T >> V >> eV >> range >> crange && i<50; i++){
        h->Fill(V);
    }
    b++;
    c2->cd(3)->cd(b);
    h->Draw((b==0)? "HIST":"SAME");
    std::cout << "RMS (" << b << ") : " << h->GetRMS() << " +- " << h->GetRMSError() << std::endl;
}while(b!=10);
*/

l3->AddEntry(hist, "Dati pre-analisi");
l3->AddEntry(ghist, "Dati pre-analisi V(t)");
l3->AddEntry(hist_corrected, "Dati senza componente");
l3->AddEntry(ghist_corrected, "Dati senza componente V(t)");
c2->cd(1);
l3->Draw();

c2->Draw();

 FCN=7401.82 FROM MIGRAD    STATUS=CONVERGED     310 CALLS         311 TOTAL
                     EDM=2.416e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.1 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           6.71176e-03   5.53512e-05  -4.46807e-08  -1.04302e+00
   2  p1           1.79162e-02   6.36272e-05   2.62674e-08   2.09250e-01
   3  p2           5.32602e-02   1.54918e-04   9.28491e-08   9.94033e-01
   4  p3           1.31560e+01   1.60539e-01   3.92796e-05  -1.01875e-03
RMS (pre) : 0.0135718 +- 0.000438486
RMS (post): 0.00393099 +- 0.000127004


Warning in <TROOT::Append>: Replacing existing TH1: hist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_corrected (Potential memory leak).


###### Alcune note finali

Andando ad eseguire un fit lineare sui primi 5/6 punti, ovvero fino a quando possiamo effettivamente osservare un regime lineare, posso trovare la velocità a cui il corpo si scalda. 

#### Valutazione errore

La curva in valutazione errore non presenta una guassiana perfetta. 
Potremmo procedere per batches, ovvero dividendo i punti, che sono effettivamente tanti, in batch da 40/50 punti e poi procedere a misurare RMS su questi valori, a questo punto poi verificarne la compatibilità e quindi verificare come di comportino singolarmente. 